<a href="https://colab.research.google.com/github/Lajs5257/DevF-MachineLearning/blob/main/Limpieza_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
import gspread
import gspread_dataframe as gd
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [18]:
nombre_rec = 'Información de contacto (Respuestas)'
worksheet = gc.open(nombre_rec).sheet1
rows = worksheet.get_all_values()
df_coments = pd.DataFrame.from_records(rows)

SpreadsheetNotFound: ignored

In [17]:
ruta = '/content/Información de contacto (Respuestas).csv'
worksheet = ruta

rows = worksheet.get_all_values()
df_coments = pd.DataFrame.from_records(rows)


AttributeError: ignored

In [15]:
df_coments.head(2)

AttributeError: ignored

In [ ]:
columnas = df_coments.iloc[0,:]
columnas

In [ ]:
df_coments.drop(0, inplace=True)

In [ ]:
df_coments.head(4)

In [ ]:
tiempo = pd.to_datetime(df_coments[0])
tiempo[:2]

1   2022-06-10 18:48:37
2   2022-06-10 18:49:07
Name: 0, dtype: datetime64[ns]

In [ ]:
# tiempo.dt.second

In [ ]:
df_coments['hora'] = tiempo.dt.hour+tiempo.dt.minute/60

In [ ]:
# df_coments.head()

In [ ]:
correos = df_coments[1].apply(lambda x: x.split('@')[1])
df_coments['correo'] = correos.apply(lambda x: 1 if x=='gmail.com' else 0)
df_coments[1][:2].tolist(),df_coments['correo'][:2].tolist()

(['adamianmm@gmail.com', 'sebastianzarraga43@aragon.unam.mx'], [1, 0])

In [ ]:
# df_coments

In [ ]:
df_coments['numero'] = df_coments[3].apply(lambda x: x if ' ' not in x else 3).replace('',0).replace('Ocho',8).astype(float)
df_coments['numero'][:2]

1     3.0
2    10.0
Name: numero, dtype: float64

In [ ]:
df_lugares = pd.get_dummies(df_coments[4].replace('Estado de México','Edo Mex').replace('Ciudad de México ','CDMX'))
df_lugares.head(2)

,BCS,CDMX,Chiapas,Ciudad de México,Durango,Edo Mex,Estado de Mexico,Guadalajara,Guanajuato,Guerrero,...,Mexico,Morelos,Nuevo León,Puebla,Querétaro,Sinaloa,Tabasco,cdmx,departamento,edo mex
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
# df_coments[5]

In [ ]:
df_coments[5][df_coments[5]!='?'].astype(int).mean()

26.08823529411765

In [ ]:
#Imputamos la media
df_coments['edad'] = df_coments[5].replace('?',26).astype(int)

In [ ]:
df_coments['genero'] = df_coments[6].apply(lambda x: 1 if x=='Hombre' else 0)

In [ ]:
df_coments[7][:2]

1    Pop, Reggaeton, Clásica, Rock, Hip Hop
2                        Reggaeton, Hip Hop
Name: 7, dtype: object

In [ ]:
listas = df_coments[7].apply(lambda x: x.split(','))
# Aplanar una lista de listas
unicos = set([item.strip().lower() for sublist in listas.tolist() for item in sublist])
unicos

{'clásica', 'hip hop', 'jazz', 'metal', 'pop', 'reggaeton', 'rock'}

In [ ]:
# listas.apply(lambda x: 1 if 'Pop' in x else 0)

In [ ]:
# listas

In [ ]:
gen = 'clásica'
# [ele.strip().lower() for ele in x] es una lista que limpia el texto ej. " Clásica" -> "clásica"
# 1 si clásica está en la lista, 0 en otro caso
listas.apply(lambda x: 1 if gen in [ele.strip().lower() for ele in x] else 0)[:2]

1    1
2    0
Name: 7, dtype: int64

In [ ]:
# Lo guardo como diccionario para que se pueda leer directamente como Data Frame
generos = {}
for gen in unicos:
  generos[gen] = listas.apply(lambda x: 1 if gen in [ele.strip().lower() for ele in x] else 0)

musica = pd.DataFrame(generos)
musica.head()

,clásica,pop,rock,hip hop,reggaeton,metal,jazz
1,1,1,1,1,1,0,0
2,0,0,0,1,1,0,0
3,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0
5,1,0,1,0,1,1,1


In [ ]:
# Pongo el prefijo mus para saber que hablo de variables de tipo musical
musica.columns = ['mus_' + x for x in musica.columns]
df_coments = pd.concat([df_coments,musica],axis=1)

In [ ]:
# Lo mismo para mascotas
listas = df_coments[8].apply(lambda x: x.split(','))
unicos = set([item.strip().lower() for sublist in listas.tolist() for item in sublist])

In [ ]:
generos = {}
for gen in unicos:
  generos[gen] = listas.apply(lambda x: 1 if gen in [ele.strip().lower() for ele in x] else 0)

musica = pd.DataFrame(generos)
musica.columns = ['masc_' + x for x in musica.columns]
df_coments = pd.concat([df_coments,musica],axis=1)

In [ ]:
df_coments.head(2)

,0,1,2,3,4,5,6,7,8,9,...,mus_rock,mus_hip hop,mus_reggaeton,mus_metal,mus_jazz,masc_,masc_peces,masc_gatos,masc_perros,masc_otros
1,6/10/2022 18:48:37,adamianmm@gmail.com,Damian,3,CDMX,22,Hombre,"Pop, Reggaeton, Clásica, Rock, Hip Hop",Gatos,"Naruto, Grey's Anatomy, Lost",...,1,1,1,0,0,0,0,1,0,0
2,6/10/2022 18:49:07,sebastianzarraga43@aragon.unam.mx,Sebastian Zaaraga,10,Mexico,22,Hombre,"Reggaeton, Hip Hop","Perros, Peces","Game of Thrones , Naruto , Sons of Anarchy",...,0,1,1,0,0,0,1,0,1,0


In [ ]:
df_coments[9]

1                          Naruto, Grey's Anatomy, Lost
2            Game of Thrones , Naruto , Sons of Anarchy
3                 Alf, príncipe de beler, prision break
4                   one piece, siete pecados capitales.
5     Mindhunter, How I Met Your Mother, Attack On T...
6                The 100, the rings of power, one piece
7         Game of Thrones, The Serpent Queen, Mr. Robot
8                 Dark, Mindhunter, The Midnight Gospel
9                       rick & morty, the 100, hannibal
10                               Reign, Suits, Millions
11              Malcolm el de enmedio, Hannibal, Dexter
12    Los chicos son mejores que las flores, La teor...
13                   Vikingos, Good Girls, Rick y Morty
14                          Dragon ball z,jojos, bleach
15               Game of Thrones, Friends, Heartstopper
16                               BCS, One Piece, Malcom
17                        D gray man, roleplay, Ladybug
18                       Gossip girl, Big Mouth,

In [ ]:
listas = df_coments[9].apply(lambda x: x.split(','))

unicos = [item.strip().lower() for sublist in listas.tolist() for item in sublist]
populares = pd.Series(unicos).value_counts().head(10).index.tolist()
populares

['game of thrones',
 'breaking bad',
 'one piece',
 'naruto',
 'the office',
 'the boys',
 'attack on titan',
 'los 100',
 'hannibal',
 'rick & morty']

In [ ]:
for i in range(len(populares)):
  if i==0:
    pops = listas.apply(lambda x: 1 if populares[i] in [ele.strip().lower() for ele in x] else 0)
  else:
    pops_ = listas.apply(lambda x: 1 if populares[i] in [ele.strip().lower() for ele in x] else 0)

    pops += pops_

df_coments['populares'] = pops

In [ ]:
# df_coments['populares']

In [ ]:
df_coments['extro_scale'] = df_coments[10].astype(int)

In [ ]:
# [c for c in df_coments.columns if c not in range(13)]

In [ ]:
#Quitamos las columnas de texto
df_coments[[c for c in df_coments.columns if c not in range(40)]].head()

,hora,correo,numero,edad,genero,mus_clásica,mus_pop,mus_rock,mus_hip hop,mus_reggaeton,mus_metal,mus_jazz,masc_,masc_peces,masc_gatos,masc_perros,masc_otros,populares,extro_scale
1,18.800000,1,3.0,22,1,1,1,1,1,1,0,0,0,0,1,0,0,1,3
2,18.816667,0,10.0,22,1,0,0,0,1,1,0,0,0,1,0,1,0,2,1
3,18.816667,0,10.0,27,1,0,0,0,1,0,0,0,0,0,1,0,0,0,2
4,18.816667,1,14.0,26,0,0,1,0,0,0,0,0,1,0,0,0,0,1,3
5,18.816667,1,13.0,21,1,1,0,1,0,1,1,1,0,0,0,1,0,1,2
